## 数据集制作

In [ ]:
# docx --> txt
from docx import Document

# 读取 .docx 文件
def docx_to_text(docx_file, txt_file):
    doc = Document(docx_file)
    with open(txt_file, 'w', encoding='utf-8') as f:
        for para in doc.paragraphs:
            f.write(para.text + '\n')

# 调用函数转换
docx_file = 'xingfa.docx'  # 你的 .docx 文件路径
txt_file = 'output.txt'    # 转换后的 .txt 文件路径
# docx_to_text(docx_file, txt_file)

print(f'已将 {docx_file} 转换为 {txt_file}')